In [38]:
import pandas as pd
import numpy as np
import os
import typing
import torch
import matplotlib.pyplot as plt
from torch_geometric_temporal.dataset import METRLADatasetLoader
from torch_geometric_temporal.signal import temporal_signal_split
import torch.utils.data as ut

In [2]:
loader = METRLADatasetLoader()

dataset = loader.get_dataset()

train_dataset, test_dataset = temporal_signal_split(dataset, train_ratio = 0.8)

In [34]:
dataset.targets[0][0]

array([0.37235928, 0.24522121, 0.49606127, 0.65206325, 0.11258511,
       0.5311103 , 0.50911874, 0.47132087, 0.42184007, 0.3909146 ,
       0.47613147, 0.56409734], dtype=float32)

In [36]:
# torch.Tensor(train_dataset.features)
#train_dataset.features[0][i][0]
batch_speed_feat = []
batch_speed_target = []
target_speed = []
for i in range(len(train_dataset.features)):
    speed = []
    target_speed = []
    for j in range(len(train_dataset.features[i])):
        speed.append(train_dataset.features[i][j][0])
        target_speed.append(train_dataset.targets[i][j])
    batch_speed_feat.append(speed)
    batch_speed_target.append(target_speed)

X = torch.Tensor(batch_speed_feat)
y = torch.Tensor(batch_speed_target)
print(X.size(), y.size())


torch.Size([27399, 207, 12]) torch.Size([27399, 207, 12])


In [96]:
dataset = ut.TensorDataset(X, y)
loader = ut.DataLoader(dataset, batch_size=16, drop_last=True)
temp = next(iter(loader))

In [75]:
train_features, train_labels = next(iter(loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([1, 207, 12])
Labels batch shape: torch.Size([1, 207, 12])


In [93]:
import torch.nn as nn
class BaselineLSTM(nn.Module):
    def __init__(self, input_dim: int, hidden_dim: int, num_layers: int):
        super().__init__()
        self.lstm = nn.LSTM(input_size=input_dim, hidden_size=hidden_dim, num_layers=num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, input_dim)
    
    def forward(self, x):
        lstm_output, _ = self.lstm(x)
        print("LSTM Output shape = ", lstm_output.shape)
        lin_out = self.linear(lstm_output)
        print("FC Output shape = ",lin_out.shape)
        return lin_out

In [94]:
import time

def train(model, dataloader, loss_func, device):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 50
    start_time = time.time()

    for idx, (label, speed) in enumerate(dataloader):
        label = label.to(device)
        speed = speed.to(device)
        optimizer.zero_grad()
        
        out = None
        ###########################################################################
        # TODO: compute the logits of the input, get the loss, and do the         #
        # gradient backpropagation.
        ###########################################################################
        out = model(speed)
        #print(out.shape)
        loss = loss_func(out, label)
        loss.backward()
        ###########################################################################
        #                             END OF YOUR CODE                            #
        ###########################################################################
        
        optimizer.step()
        
        train_rmse = torch.sqrt(loss)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| rmse {:8.3f}'.format(epoch, idx, len(dataloader),
                                              train_rmse))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(model, dataloader, loss_func, device):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text) in enumerate(dataloader):
            label = label.to(device)
            text = text.to(device)
            
            ###########################################################################
            # TODO: compute the logits of the input, get the loss.                    #
            ###########################################################################
            logits = model(text)
            loss = loss_func(logits, label)
            ###########################################################################
            #                             END OF YOUR CODE                            #
            ###########################################################################
            
            total_acc += (logits.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

In [97]:
from torch.utils.data.dataset import random_split
import torch.nn.functional as F
import time 

assert torch.cuda.is_available()
# device = 'cuda'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
# Hyper parameters
epochs = 1 # epoch
lr = 0.0005 # learning rate
batch_size = 1 # batch size for training
input_size = 12
hidden_dim = 64


###########################################################################
# TODO: Deinfe the classifier and loss function.
###########################################################################
model = BaselineLSTM(input_size, hidden_dim, num_layers=2)
loss_func = F.mse_loss #nn.CrossEntropyLoss()#reduction='sum'
###########################################################################
#                             END OF YOUR CODE                            #
###########################################################################

# copy the model to the specified device (GPU)
model = model.to(device)
        
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, epochs, 1e-8)
total_accu = None


for epoch in range(1, epochs + 1):
    print(epoch)
    epoch_start_time = time.time()
    train(model, loader, loss_func, device)
    # accu_val = evaluate(model, valid_dataloader, loss_func, device)
    # if total_accu is not None and total_accu > accu_val:
    #     scheduler.step()
    # else:
    #     total_accu = accu_val
    # print('-' * 59)
    # print('| end of epoch {:3d} | time: {:5.2f}s | '
    #       'valid accuracy {:8.3f} '.format(epoch,
    #                                        time.time() - epoch_start_time,
    #                                        accu_val))
    # print('-' * 59)

cuda
1
LSTM Output shape =  torch.Size([16, 207, 64])
FC Output shape =  torch.Size([16, 207, 12])
LSTM Output shape =  torch.Size([16, 207, 64])
FC Output shape =  torch.Size([16, 207, 12])
LSTM Output shape =  torch.Size([16, 207, 64])
FC Output shape =  torch.Size([16, 207, 12])
LSTM Output shape =  torch.Size([16, 207, 64])
FC Output shape =  torch.Size([16, 207, 12])
LSTM Output shape =  torch.Size([16, 207, 64])
FC Output shape =  torch.Size([16, 207, 12])
LSTM Output shape =  torch.Size([16, 207, 64])
FC Output shape =  torch.Size([16, 207, 12])
LSTM Output shape =  torch.Size([16, 207, 64])
FC Output shape =  torch.Size([16, 207, 12])
LSTM Output shape =  torch.Size([16, 207, 64])
FC Output shape =  torch.Size([16, 207, 12])
LSTM Output shape =  torch.Size([16, 207, 64])
FC Output shape =  torch.Size([16, 207, 12])
LSTM Output shape =  torch.Size([16, 207, 64])
FC Output shape =  torch.Size([16, 207, 12])
LSTM Output shape =  torch.Size([16, 207, 64])
FC Output shape =  torch.S

In [65]:
torch.cuda.get_device_name(0)

'NVIDIA GeForce RTX 3060 Laptop GPU'